In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from MotionBERT.common_import import *
os.chdir('data/motion3d/kookmin')

In [2]:
cam_param_kookmin = readpkl('cam_param_kookmin.pkl')
cam_param_kookmin.keys()

dict_keys(['for_pnp_data', 'S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10'])

In [3]:
cam_param_kookmin['S01'].keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8])

In [4]:
cam_param_kookmin['S01'][1].keys()

dict_keys(['H', 'W', 'intrinsic', 'R', 't', 'C', 'pan', 'tilt'])

In [5]:
pkl.keys()

NameError: name 'pkl' is not defined

In [6]:
pkl_list = glob('kookmin_pkl_with_kookmin_setting2/*.pkl')
cam_3ds = {}
img_2ds = {}
img_3ds = {}
img_25ds = {}
scale_factors = {}
source_list = []
cam_params = {}
for pkl_path in tqdm(pkl_list):
    #print(pkl_path)
    pkl = readpkl(pkl_path)
    subject = pkl_path.split('/')[-1].split('_')[0]
    cam_id = pkl_path.split('/')[-1].split('_')[1]
    action = pkl_path.split('/')[-1].split('_')[2].split('.')[0]
    source_list.append(f'{subject}_{cam_id}_{action}')
    #print(subject, action, cam_id)
    if subject not in cam_3ds:
        cam_3ds[subject] = {}
        img_2ds[subject] = {}
        img_3ds[subject] = {}
        img_25ds[subject] = {}
        scale_factors[subject] = {}
        cam_params[subject] = {}
    if action not in cam_3ds[subject]:
        cam_3ds[subject][action] = {}
        img_2ds[subject][action] = {}
        img_3ds[subject][action] = {}
        img_25ds[subject][action] = {}
        scale_factors[subject][action] = {}
        cam_params[subject][action] = {}
    #print(pkl.keys())
    cam_3d = np.array(pkl['cam_3d'])/1000 # in m
    img_2d = np.array(pkl['joint_2d'])
    img_3d = np.array(pkl['joint3d_image'])
    img_25d = np.array(pkl['joints_2.5d_image'])
    scale_factor = np.array(pkl['2.5d_factor'])
    cam_3ds[subject][action][cam_id] = cam_3d
    img_2ds[subject][action][cam_id] = img_2d
    img_3ds[subject][action][cam_id] = img_3d
    img_25ds[subject][action][cam_id] = img_25d
    scale_factors[subject][action][cam_id] = scale_factor

    num_frames = len(cam_3d)
    intrinsic = pkl['cam_param'][0].tolist()['intrinsic']
    W = intrinsic[0, 2] * 2
    H = intrinsic[1, 2] * 2
    R = pkl['cam_param'][0].tolist()['extrinsic'][:, :3]
    t = pkl['cam_param'][0].tolist()['extrinsic'][:, 3]
    C = - R.T @ t
    extrinsic = np.hstack([R, t.reshape(-1, 1)])
    cam_params[subject][action][cam_id] = {
        'intrinsic': intrinsic, 'extrinsic': extrinsic, 'C': C, 'R': R, 't': t, 'W': W, 'H': H, 'num_frames': num_frames
    }
    #break


  0%|          | 0/350 [00:00<?, ?it/s]

100%|██████████| 350/350 [00:17<00:00, 19.90it/s]


In [14]:
img_25ds[subject][action][cam_id][0], img_3ds[subject][action][cam_id][0] * scale_factors[subject][action][cam_id][0]

(array([[ 682.38822197, 1490.55057702,    0.        ],
        [ 592.75639947, 1493.10101264,   30.17413821],
        [ 602.91712396, 1853.88803034,   94.88104973],
        [ 633.17302359, 2206.34978378,  208.96144169],
        [ 773.63173546, 1487.95428142,  -30.17413821],
        [ 798.75222803, 1877.1035902 ,   36.73465517],
        [ 836.05348963, 2224.48255637,  149.28730539],
        [ 643.49456276, 1184.63961866, -121.1853596 ],
        [ 651.17812447,  995.29462655, -127.46673481],
        [ 620.80439764,  798.52687312, -225.25690118],
        [ 660.17522926,  640.62676584,  -94.35227406],
        [ 843.98226954, 1000.73352055, -112.99235012],
        [ 905.08767011, 1296.84093526,  -63.61193382],
        [ 882.58912791, 1548.28669269, -123.33493299],
        [ 510.9119814 , 1011.23440389,  -10.74314275],
        [ 473.60893533, 1296.1397554 ,   78.01322677],
        [ 429.1119741 , 1535.50949327,   35.99575935]]),
 array([[ 682.38822197, 1490.55057702,    0.        ],
        

In [42]:
save_path_cam_3d = 'kookmin-cam_3d.pkl'
save_path_img_2d = 'kookmin-img_2d.pkl'
save_path_source_list = 'kookmin-source_list.pkl'
save_path_img_3d = 'kookmin-img_3d.pkl'
save_path_img_25d = 'kookmin-img_25d.pkl'
save_path_scale_factor = 'kookmin-scale_factor.pkl'
save_path_cam_param = 'kookmin-cam_param.pkl'
savepkl(cam_3ds, save_path_cam_3d)
savepkl(img_2ds, save_path_img_2d)
savepkl(source_list, save_path_source_list)
savepkl(img_3ds, save_path_img_3d)
savepkl(img_25ds, save_path_img_25d)
savepkl(scale_factors, save_path_scale_factor)
savepkl(cam_params, save_path_cam_param)

In [12]:
source_list

['S12_5_yoga',
 'S19_1_benchpress',
 'S13_2_benchpress',
 'S15_5_burpee',
 'S20_5_burpee',
 'S19_3_kneeup',
 'S19_5_cat',
 'S11_4_squat',
 'S16_2_yoga',
 'S14_5_pushup',
 'S18_3_yoga',
 'S14_1_benchpress',
 'S20_2_yoga',
 'S11_4_cat',
 'S17_2_pushup',
 'S13_3_kneeup',
 'S19_5_squat',
 'S20_5_kneeup',
 'S19_4_burpee',
 'S19_4_pushup',
 'S11_5_pushup',
 'S13_4_squat',
 'S20_2_burpee',
 'S13_3_burpee',
 'S20_2_benchpress',
 'S16_3_pushup',
 'S20_3_burpee',
 'S13_5_yoga',
 'S15_3_cat',
 'S18_1_benchpress',
 'S11_1_yoga',
 'S16_4_cat',
 'S13_4_benchpress',
 'S17_1_kneeup',
 'S20_1_kneeup',
 'S14_4_yoga',
 'S19_4_cat',
 'S14_3_kneeup',
 'S12_2_burpee',
 'S20_2_kneeup',
 'S13_3_yoga',
 'S20_4_benchpress',
 'S19_1_squat',
 'S18_4_burpee',
 'S17_3_yoga',
 'S14_2_benchpress',
 'S14_3_benchpress',
 'S17_1_squat',
 'S11_1_kneeup',
 'S18_1_kneeup',
 'S13_5_pushup',
 'S13_2_pushup',
 'S17_5_burpee',
 'S18_5_burpee',
 'S20_1_benchpress',
 'S11_2_benchpress',
 'S15_1_squat',
 'S17_2_burpee',
 'S20_1_b

In [11]:
data_type_list = ['source_list', 'cam_3d', 'img_2d', 'img_3d', 'img_25d', 'scale_factor', 'cam_param']
overwrite_list = []
dataset_name = 'kookmin'
data_aug = default_data_aug.copy()
data = load_data_dict(dataset_name, data_type_list, overwrite_list, verbose=True, data_aug=data_aug)

[overwrite: False] ==> Loading KOOKMIN source_list...
[overwrite: False] ==> Loading KOOKMIN cam_3d...
[overwrite: False] ==> Loading KOOKMIN img_2d...
[overwrite: False] ==> Loading KOOKMIN img_3d...
[overwrite: False] ==> Loading KOOKMIN img_25d...
[overwrite: False] ==> Loading KOOKMIN scale_factor...
[overwrite: False] ==> Loading KOOKMIN cam_param...
